In [ ]:
import json
import os
import csv
import math
import numpy as np
import pandas as pd

from automates.model_assembly.networks import GroundedFunctionNetwork
from automates.model_analysis.sensitivity import (
    SensitivityAnalyzer,
    SADependentVariable,
)

In this first section, variable bounds, fixed inputs, and outputs are defined.

In [ ]:
def gather_stemp_soilt_inputs():
    bounds = {
        "stemp_soilt::stemp_soilt.soilt::srad::-1": [0.5, 30],
        "stemp_soilt::stemp_soilt.soilt::tmax::-1": [0, 40],
        # "stemp_soilt::stemp_soilt.soilt::nlayr::-1": [1, 20],
    }

    inputs = {
        # "stemp_soilt::stemp_soilt.soilt::nlayr::-1": SADependentVariable(
        #     "stemp_soilt::stemp_soilt.soilt::nlayr::-1",
        #     ["stemp_soilt::stemp_soilt.soilt::nlayr::-1"],
        #     lambda nlayr: int(nlayr),
        # ),
        "stemp_soilt::stemp_soilt.soilt::nlayr::-1": 10,
        "stemp_soilt::stemp_soilt.soilt::atot::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::tma::-1": [0, 0, 0, 0, 0],
        "stemp_soilt::stemp_soilt.soilt::ww::-1": 1,
        "stemp_soilt::stemp_soilt.soilt::albedo::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::b::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::cumdpt::-1": 1,
        "stemp_soilt::stemp_soilt.soilt::doy::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::dp::-1": 1,
        "stemp_soilt::stemp_soilt.soilt::hday::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::pesw::-1": 1,
        "stemp_soilt::stemp_soilt.soilt::tamp::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::tav::-1": SADependentVariable(
            "stemp_soilt::stemp_soilt.soilt::tav::-1",
            ["stemp_soilt::stemp_soilt.soilt::tmax::-1"],
            lambda tmax: tmax - 5,
        ),
        "stemp_soilt::stemp_soilt.soilt::tavg::-1": SADependentVariable(
            "stemp_soilt::stemp_soilt.soilt::tavg::-1",
            [
                "stemp_soilt::stemp_soilt.soilt::tmax::-1",
                "stemp_soilt::stemp_soilt.soilt::tav::-1",
            ],
            lambda tmax, tav: (tmax + tav) / 2,
        ),
        "stemp_soilt::stemp_soilt.soilt::dsmid::-1": SADependentVariable(
            "stemp_soilt::stemp_soilt.soilt::dsmid::-1",
            ["stemp_soilt::stemp_soilt.soilt::nlayr::-1"],
            lambda nlayr: [0] * nlayr,
        ),
        "stemp_soilt::stemp_soilt.soilt::st::-1": SADependentVariable(
            "stemp_soilt::stemp_soilt.soilt::st::-1",
            ["stemp_soilt::stemp_soilt.soilt::nlayr::-1"],
            lambda nlayr: [0] * nlayr,
        ),
    }

    output_base_names = ["atot", "tma", "srftemp", "st"]
    # output_base_names = ["tma"]

    return (bounds, inputs, output_base_names)


def gather_stemp_epic_soilt_inputs():
    bounds = {
        # "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::srad::-1": [0.5, 30],
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmax::-1": [0, 40],
        # "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1": [1, 20],
    }

    inputs = {
        # "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1": SADependentVariable(
        #     "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1",
        #     ["stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1"],
        #     lambda nlayr: int(nlayr),
        # ),
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::b::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::bcv::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1": 10,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tma::-1": [0, 0, 0, 0, 0],
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::ww::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::b::-1": 0,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::cumdpt::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::dp::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::pesw::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::wetday::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::wft::-1": 20,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::ww::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tav::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmin::-1": SADependentVariable(
            "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmin::-1",
            ["stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmax::-1"],
            lambda tmax: tmax - 5,
        ),
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tavg::-1": SADependentVariable(
            "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tavg::-1",
            [
                "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmax::-1",
                "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tav::-1",
            ],
            lambda tmax, tav: (tmax + tav) / 2,
        ),
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::dsmid::-1": SADependentVariable(
            "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::dsmid::-1",
            ["stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1"],
            lambda nlayr: [0] * nlayr,
        ),
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::st::-1": SADependentVariable(
            "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::st::-1",
            ["stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1"],
            lambda nlayr: [0] * nlayr,
        ),
    }

    output_base_names = ["tma", "srftemp", "st", "x2_avg"]

    return (bounds, inputs, output_base_names)

Next, the location of the GrFN files and $N$, the number of samples to analyze when generating Si, and the option to compute second-order indices.

In [ ]:
N = 1000 
compute_S2 = False #Leave as true for now
file = "stemp_soilt--GrFN.json"  # file location
file_epic = "stemp_epic_soilt--GrFN.json"  # epic file location

The sensitivity analysis follows.

In [ ]:
if file is not None:
    print("Reading soilt json file...")
    if not os.path.exists(file):
        raise Exception(f"Error: File '{file}' does not exist")
    (bounds, inputs, outputs) = gather_stemp_soilt_inputs()
    
    print("Parsing grfn json...")
    grfn = None
    try:
        grfn = GroundedFunctionNetwork.from_json(file)
    except Exception:
        raise Exception(f"Error: Unable to process grfn json file {file}")
        
    print("Running sensitivity analysis...")
    analyzer = SensitivityAnalyzer()
    Si_list = analyzer.Si_from_Sobol(N, grfn, bounds, inputs, outputs, calc_2nd=compute_S2)
    model_inputs = [
        {
            "variable_name": k,
            "bounds": v,
        }
        for k, v in bounds.items()
    ]
    model_outputs = [{"variable_name": n} for n in outputs]
    results = {
        "grfn_uid": grfn.uid,
        "input_variables": model_inputs,
        "output_variables": model_outputs,
        "sensitivity_indices": [Si.to_dict() for Si in Si_list],
    }

In [ ]:
if file_epic is not None:
    print("Reading soilt epic json file...")
    if not os.path.exists(file_epic):
        raise Exception(f"Error: File '{file_epic}' does not exist")
    (bounds_epic, inputs_epic, outputs_epic) = gather_stemp_epic_soilt_inputs()
    
    print("Parsing grfn_epic json...")
    grfn_epic = None
    try:
        grfn_epic = GroundedFunctionNetwork.from_json(file_epic)
    except Exception:
        raise Exception(f"Error: Unable to process grfn_epic json file {file_epic}")
        
    print("Running epic sensitivity analysis...")
    analyzer = SensitivityAnalyzer()
    Si_list_epic = analyzer.Si_from_Sobol(N, grfn_epic, bounds_epic, inputs_epic, outputs_epic, calc_2nd=compute_s2)
    model_inputs_epic = [
        {
            "variable_name": k,
            "bounds": v,
        }
        for k, v in bounds_epic.items()
    ]
    model_outputs_epic = [{"variable_name": n} for n in outputs_epic]
    results_epic = {
        "grfn_uid": grfn_epic.uid,
        "input_variables": model_inputs_epic,
        "output_variables": model_outputs_epic,
        "sensitivity_indices": [Si.to_dict() for Si in Si_list_epic],
    }

The results are displayed.

In [ ]:
def to_table(csv_table): 
    old_table = np.array(csv_table)
    col_names = old_table[0,:]
    row_names = old_table[:,0]

    new_table = old_table[1:len(row_names), 1:len(col_names)]
    col_names = np.delete(col_names, 0)
    row_names = np.delete(row_names, 0)
    
    return pd.DataFrame(new_table, columns=col_names, index=row_names)

def pd_tables(sa_obj):
#     json_file = file

#     sa_obj = {}
#     with open(json_file) as f:
#         sa_obj = json.load(f)

    outputs = []
    inputs = []

    for v in sa_obj["output_variables"]:
        outputs.append(v["variable_name"])

    for v in sa_obj["input_variables"]:
        identifier = v["variable_name"]
        name = identifier.split("::")[-2]
        inputs.append(name)

    table = [[""] * (len(inputs) + 1) for v in range(len(outputs) + 1)]

    for col in range(len(inputs)):
        table[0][col + 1] = inputs[col]

    for idx,_ in enumerate(outputs):
        si = sa_obj['sensitivity_indices'][idx]
        table[idx + 1][0] = outputs[idx]
        for i in range(len(si['S1'])):
            S1_val = "--" if si['S1'][i] is None or math.isnan(si['S1'][i]) else str(si['S1'][i]).split(".")[0] + "." + str(si['S1'][i]).split(".")[1][:3]
            S1_conf = "--" if si['S1_conf'][i] is None or math.isnan(si['S1_conf'][i]) else str(si['S1_conf'][i]).split(".")[0] + "." + str(si['S1_conf'][i]).split(".")[1][:3]

            table[idx + 1][i + 1] = f"{S1_val} +/- {S1_conf}"
            
    # ST Table
    st_table = [[""] * (len(inputs) + 1) for v in range(len(outputs) + 1)]

    for col in range(len(inputs)):
        st_table[0][col + 1] = inputs[col]

    for idx,_ in enumerate(outputs):
        si = sa_obj['sensitivity_indices'][idx]
        st_table[idx + 1][0] = outputs[idx]
        for i in range(len(si['ST'])):
            ST_val = "--" if si['ST'][i] is None or math.isnan(si['ST'][i]) else str(si['ST'][i]).split(".")[0] + "." + str(si['ST'][i]).split(".")[1][:3]
            ST_conf = "--" if si['ST_conf'][i] is None or math.isnan(si['ST_conf'][i]) else str(si['ST_conf'][i]).split(".")[0] + "." + str(si['ST_conf'][i]).split(".")[1][:3]

            st_table[idx + 1][i + 1] = f"{ST_val} +/- {ST_conf}"

    # S2 Table
    if compute_s2:
        n_inputs = len(inputs)
        n_cols = math.comb(n_inputs, 2)
        s2_table = [[""] * (n_cols + 1) for v in range(len(outputs) + 1)]
        s2_columns = [""]
        for i in range(len(inputs)):
            for j in range(i+1, n_inputs):
                s2_columns.append(f"{inputs[i]}, {inputs[j]}")
        s2_table[0] =  s2_columns

        for idx,_ in enumerate(outputs):
            si = sa_obj['sensitivity_indices'][idx]
            s2_raw = si['S2']
            s2_raw_conf = si['S2_conf']
            s2_trimmed = [outputs[idx]]
            for i in range(n_inputs):
                for j in range(i+1, n_inputs):
                    s2_val = "--" if s2_raw[i][j] is None or math.isnan(s2_raw[i][j]) else str(s2_raw[i][j]).split(".")[0] + "." + str(s2_raw[i][j]).split(".")[1][:3]
                    s2_conf = "--" if s2_raw_conf[i][j] is None or math.isnan(s2_raw_conf[i][j]) else str(s2_raw_conf[i][j]).split(".")[0] + "." + str(s2_raw_conf[i][j]).split(".")[1][:3]
                    s2_trimmed.append(f"{s2_val} +/- {s2_conf}")
            s2_table[idx + 1] = s2_trimmed
        return (to_table(table), to_table(s2_table), to_table(st_table))
    return (to_table(table), to_table(st_table))
            



#     csv_file_name = f"{json_file.rsplit('.', 1)[0]}.csv"
#     with open(csv_file_name, "w") as csv_file:
#         csvWriter = csv.writer(csv_file, delimiter=',')
#         csvWriter.writerows(table)







In [ ]:
results['sensitivity_indices']

In [ ]:
if file is not None:
#     out_file = f"soilt_model-{N}--SA.json"
#     print(f"Writing results to {out_file}")
#     json.dump(results, open(out_file, "w"))
#     csv_result = to_csv(results)
    if compute_s2:
        (s1, s2, st) = pd_tables(results)
    else:
        (s1, st) = pd_tables(results)
if file_epic is not None:
#     out_file_epic = f"soilt_epic_model-{N}--SA.json"
#     print(f"Writing results to {out_file_epic}")
#     json.dump(results, open(out_file_epic, "w"))
#     csv_epic_result = to_csv(results_epic)
    if compute_s2:
        (s1_epic, s2_epic, st_epic) = pd_tables(results_epic)
    else:
        (s1_epic, st_epic) = pd_tables(results_epic)

In [ ]:
s1

In [ ]:
s1_epic

In [ ]:
st

In [ ]:
st_epic

In [ ]:
s2

In [ ]:
s2_epic